In [1]:
# Copyright (c) 2021 The MATCH Authors. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np
from switch_model.reporting.report_functions import *
import pickle

# TODO: remove input folder = '../../MODEL_RUNS/test_PCE/inputs/hourly_90'

cambium_dir = Path.cwd() / '../../MODEL_RUNS/nrel_cambium'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

year = 2025

In [2]:
year = 2025
def load_cambium_data(cambium_dir, scenario, year):

    # if the year is an even number, load the file corresponding to the year
    if year % 2 == 0:
        cambium = pd.read_csv(cambium_dir / f'StdScen20_{scenario}_hourly_CA_{year}.csv', skiprows=2)

        cambium.index = pd.to_datetime(cambium.timestamp_local)

        cambium = cambium.drop(columns=['timestamp','timestamp_local'])
    # otherwise, average together the two years on either side of the year
    else:
        firstyear = year - 1
        secondyear = year + 1

        cambium_1 = pd.read_csv(cambium_dir / f'StdScen20_MidCase_hourly_CA_{firstyear}.csv', skiprows=2)
        cambium_2 = pd.read_csv(cambium_dir / f'StdScen20_MidCase_hourly_CA_{secondyear}.csv', skiprows=2)

        # get the datetimeindex
        cambium_1

        cambium = (cambium_1.drop(columns=['timestamp','timestamp_local']) + cambium_2.drop(columns=['timestamp','timestamp_local'])) / 2

        cambium.index = pd.DatetimeIndex(pd.date_range(start=f'01/01/{year} 00:00', end=f'12/31/{year} 23:00', freq='H', name='timestamp_local'))
    return cambium


cambium_low = load_cambium_data(cambium_dir, 'LowRECost', year)
cambium_mid = load_cambium_data(cambium_dir, 'MidCase', year)
cambium_high = load_cambium_data(cambium_dir, 'HighRECost', year)

In [75]:
region = 'CAMXc'

def load_cambium_lrmer(cambium_dir, scenario, region):
    
    lrmer = pd.read_excel(cambium_dir / f'Cambium_LRMER_StdScen20_{scenario}.xlsx', sheet_name=region,  skiprows=8).drop(columns='Hour of the year')

    multiplier = pd.read_excel(cambium_dir / f'Cambium_LRMER_StdScen20_{scenario}.xlsx', sheet_name='CO2 to CO2e Multipliers', skiprows=1, usecols='B:C')

    multiplier = multiplier.loc[multiplier['GEA Region'] == region, 'CO2 to CO2e Multiplier'].item()

    return lrmer, multiplier



In [78]:
def calculate_levelized_lrmer(region, start_year, period, discount, unit:['CO2','CO2e']):
    
    scenarios = ['LowRECost','MidCase','HighRECost']

    levelized_lrmer = pd.DataFrame()

    for scenario in scenarios:

        lrmer, multiplier = load_cambium_lrmer(cambium_dir, scenario, region)

        # start a variable to track the discounted number of years for the levelization
        denominator = 0

        # for each year in the lrmer columns calculate the weighting
        for year in lrmer.columns:
            if (start_year <= year-1) & (start_year + period > year-1):
                odd_year_weight = 1/((1+discount)**(year-1-start_year))
            else:
                odd_year_weight = 0
            if (start_year <= year) & (start_year + period > year):
                even_year_weight = 1/((1+discount)**(year-start_year))
            else:
                even_year_weight = 0
            total_weight = odd_year_weight + even_year_weight
            
            denominator += total_weight

            # multiply the single year lrmer data by the weighting factor
            lrmer[year] = lrmer[year] * total_weight

        # calculate the levelized lrmer
        levelized_lrmer_s = lrmer.sum(axis=1) / denominator

        # adjust for the unit
        if unit == 'CO2e':
            levelized_lrmer_s = levelized_lrmer_s * multiplier

        levelized_lrmer[scenario] = levelized_lrmer_s

    return levelized_lrmer


levelized_lrmer = calculate_levelized_lrmer(region, year, 20, 0.03, 'CO2e')


In [79]:
levelized_lrmer

,LowRECost,MidCase,HighRECost
0,126.27,168.35,197.68
1,134.73,174.87,199.26
2,142.36,175.57,204.13
3,141.84,182.70,203.65
4,137.34,178.54,194.27
...,...,...,...
8755,104.57,133.89,155.78
8756,103.16,136.93,157.61
8757,109.79,138.27,169.42
8758,109.19,152.46,177.76


In [3]:
# calculate system metrics

def calculate_system_peak(cambium, addl_dispatch, addl_storage_dispatch):

    net_load = cambium.copy()[['net_load_busbar']]
    #net_load.index = pd.to_datetime(net_load.index)

    # find the maximum net load on each day
    peak_demand = net_load.groupby(net_load.index.date).max()

    # find the average daily peak load in each quarter
    peak_demand.index = pd.to_datetime(peak_demand.index)
    peak_demand = peak_demand.groupby(peak_demand.index.quarter).mean()

    # find the hour of each day when net load peaks
    peak_hour = net_load.groupby(net_load.index.date).idxmax()
    peak_hour['net_load_busbar'] = peak_hour['net_load_busbar'].dt.hour

    # find the average hour during which net load peaks in each quarter
    peak_hour.index = pd.to_datetime(peak_hour.index)
    peak_hour = peak_hour.groupby(peak_hour.index.quarter).mean()
    peak_hour['peak_hour'] = peak_hour['net_load_busbar'].apply(lambda row: f'{int(row)}:{int((row*60)%60):02d}')

    #combine the data together
    peak = peak_demand.rename(columns={'net_load_busbar':'net_load_peak_MW'})
    peak['peak_hour'] = peak_hour['peak_hour']

    return peak

calculate_system_peak(cambium_mid)

,net_load_peak_MW,peak_hour
1,"33,846.93",18:29
2,"34,054.96",19:58
3,"42,228.82",18:56
4,"36,270.86",18:01


In [65]:
# calculate daily 3 hour ramp

ramp_length = 3

def calculate_system_ramp(cambium, ramp_length):

    net_load = cambium.copy()[['net_load_busbar']]
    #net_load.index = pd.to_datetime(net_load.index)

    ramp = net_load.shift(-ramp_length, fill_value=0) - net_load
    ramp.index = pd.to_datetime(ramp.index)
    max_ramp = ramp.groupby([ramp.index.date]).max()
    max_ramp.index = pd.to_datetime(max_ramp.index)
    max_ramp = max_ramp.groupby(max_ramp.index.quarter).mean()


    # find the hour of each day when net load peaks
    max_ramp_hour = ramp.groupby([ramp.index.date]).idxmax()
    max_ramp_hour['net_load_busbar'] = max_ramp_hour['net_load_busbar'].dt.hour

    # find the average hour during which net load peaks in each quarter
    max_ramp_hour.index = pd.to_datetime(max_ramp_hour.index)
    max_ramp_hour = max_ramp_hour.groupby(max_ramp_hour.index.quarter).mean()
    max_ramp_hour['max_ramp_hour'] = max_ramp_hour['net_load_busbar'].apply(lambda row: f'{int(row)}:{int((row*60)%60):02d}')

    #combine the data together
    ramp = max_ramp.rename(columns={'net_load_busbar':f'max_{ramp_length}_hr_ramp_MW'})
    ramp['ramp_start_hour'] = max_ramp_hour['max_ramp_hour']

    return ramp

calculate_system_ramp(cambium, ramp_length)   

,max_3_hr_ramp_MW,ramp_start_hour
1,"20,167.99",14:37
2,"19,434.80",15:32
3,"16,807.00",15:01
4,"17,839.90",14:03


In [68]:
def calculate_residual_mix(cambium):
    """
    Still need to convert units to lb/MWh
    """
    resid_mix = cambium.copy()[['co2_rate_avg_gen','generation', 'coal_MWh','coal-ccs_MWh','o-g-s_MWh','gas-cc_MWh','gas-cc-ccs_MWh','gas-ct_MWh']]

    resid_mix['residual_generation'] = resid_mix[['coal_MWh','coal-ccs_MWh','o-g-s_MWh','gas-cc_MWh','gas-cc-ccs_MWh','gas-ct_MWh']].sum(axis=1)
    resid_mix['co2_rate_residual'] = (resid_mix.co2_rate_avg_gen * resid_mix.generation) / resid_mix.residual_generation

    resid_mix = resid_mix[['co2_rate_residual']]

    return resid_mix

calculate_residual_mix(cambium)

,co2_rate_residual
timestamp_local,
2025-01-01 00:00:00,399.74
2025-01-01 01:00:00,398.59
2025-01-01 02:00:00,398.60
2025-01-01 03:00:00,398.19
2025-01-01 04:00:00,399.08
...,...
2025-12-31 19:00:00,416.61
2025-12-31 20:00:00,408.18
2025-12-31 21:00:00,426.73
